# 7_Controlling_Bean_Selection

When multiple implementations of an interface exist in Spring Boot, the framework needs to know **which bean to inject**. This notebook will explain how to control bean selection using annotations like `@Primary` and `@Qualifier` — with crystal clear examples and explanations. 🌈

---

## ⚠️ The Problem: Multiple Beans, One Interface

Let’s say you have this interface:

```java
public interface PaymentService {
    void processPayment(double amount);
}
```

And you have two implementations:

```java
@Service
public class PaypalPaymentService implements PaymentService {
    ...
}
```

```java
@Service
public class StripePaymentService implements PaymentService {
    ...
}
```

### ❗ What Happens?

When Spring tries to inject a `PaymentService` into `OrderService`, it sees **two beans** of the same type. It gets **confused** and throws:

### 🚫 Error:

```
Parameter 0 of constructor in OrderService required a single bean, but 2 were found:
- paypalPaymentService
- stripePaymentService
```

👉 This means: Spring can't decide which one to inject!

---

## ✅ Solution 1: `@Primary`

Use `@Primary` to tell Spring which bean to prefer when multiple are available.

### 📦 `PaypalPaymentService.java`

```java
package com.codewithsithum.store;

import org.springframework.context.annotation.Primary;
import org.springframework.stereotype.Service;

@Service
@Primary // ✅ This tells Spring: use this when no @Qualifier is specified
public class PaypalPaymentService implements PaymentService {
    @Override
    public void processPayment(double amount) {
        System.out.println("Paypal");
        System.out.println("Amount: " + amount);
    }
}
```

With this setup, Spring will inject `PaypalPaymentService` automatically into any `PaymentService` dependency **unless otherwise specified**.

✅ Advantage: Simple and elegant default bean selection.

---

## ✅ Solution 2: `@Qualifier`

Use `@Qualifier` to **explicitly name** the bean you want to inject.

### 📦 `StripePaymentService.java`

```java
package com.codewithsithum.store;

import org.springframework.stereotype.Service;

@Service("stripe") // 🏷️ Named bean
public class StripePaymentService implements PaymentService {
    @Override
    public void processPayment(double amount){
        System.out.println("STRIPE");
        System.out.println("Amount: " + amount);
    }
}
```

### 📦 `PaypalPaymentService.java`

```java
package com.codewithsithum.store;

import org.springframework.stereotype.Service;

@Service("paypal") // 🏷️ Named bean
public class PaypalPaymentService implements PaymentService {
    @Override
    public void processPayment(double amount) {
        System.out.println("Paypal");
        System.out.println("Amount: " + amount);
    }
}
```

### 📦 `OrderService.java`

```java
package com.codewithsithum.store;

import org.springframework.beans.factory.annotation.Qualifier;
import org.springframework.stereotype.Service;

@Service
public class OrderService {

    private PaymentService paymentService;

    // 💉 Constructor Injection with @Qualifier
    public OrderService(@Qualifier("paypal") PaymentService paymentService){
        this.paymentService = paymentService;
    }

    public void placeOrder(){
        paymentService.processPayment(10);
    }

    // Optional: Setter injection
    public void setPaymentService(PaymentService paymentService) {
        this.paymentService = paymentService;
    }
}
```

🎯 Now, Spring **explicitly knows** to use the `paypal` bean for `OrderService`.

---

## 🧠 Summary Table

| Use Case                         | Annotation                       | Result                             |
| -------------------------------- | -------------------------------- | ---------------------------------- |
| You have only one implementation | None needed                      | ✅ Works fine                       |
| You want a default among many    | `@Primary` on the preferred bean | ✅ Spring uses it unless overridden |
| You want full control            | `@Qualifier("beanName")`         | ✅ Choose specific bean             |

---

## 🏁 Final Thoughts

* Use `@Primary` for sensible defaults
* Use `@Qualifier` for explicit control
* Avoid bean confusion by **naming** your services clearly

🎉 Now you have full control over which beans are selected in Spring Boot!
